# MICCAI 2023 Tutorial - Part I:
## Launch a containerized medical imaging application
This Notebook is designed to reproduce the computational experiments of a [published scientific paper](https://hal.science/hal-04006057) on deep learning for **Bra**in **T**umor **S**egmentation (**BraTS**).

<img src="imgs/BraTS-Results.png" alt="BraTS-Results" height="250" title="Results from Brain Tumor Segmentation"/>

It uses methods from the [VIP client for Python](https://github.com/virtual-imaging-platform/VIP-python-client).

*__N.B.__: This tutorial is designed as a team work with other attendees. __Please wait for the speakers' instructions when required__ by the italic-style cell comments*.

---
## Useful libraries

**Install step**: uncomment & run the following cell *only* if you are using *Google Colab* (not *Binder*)

In [ ]:
# Remove the hashtag (#) only
#! pip install vip-client girder-client

**Importation step**

In [ ]:
# Builtins
import json
from pathlib import *
from random import choice
# Installed
from vip_client.classes import VipLauncher
from vip_client.classes import VipLoader

---
## Prepare your executions

Handshake with VIP using your **API key** 

*Follow this [procedure](../../doc/account.md) to get a VIP API key from the [VIP portal](https://vip.creatis.insa-lyon.fr/).*

In [ ]:
# Paste your VIP API key between the quotes
VipLauncher.init(api_key="VIP_API_KEY");

Show the (small) dataset that will be used in this tutorial

In [ ]:
# The data you need is stored in a folder on VIP Servers 
input_dir = PurePosixPath("/vip/MICCAI tutorial (group)/inputs")
# This folder contains data from several subjects
subjects = VipLoader.list_dir(input_dir)
print("\n".join(subjects))

Show the applications that will be used in this tutorial 

In [ ]:
VipLauncher.show_pipeline("BratsPipeline/")

Application `BraTSPipeline` is declined in two different versions. You will be able to launch one of them.

*Please let chance choose your version to get a balanced dataset within your team*

In [ ]:
# Available versions
pipelines = {
    "v181": "BraTSPipeline/1.8.1_cvmfs", 
    "v190": "BraTSPipeline/1.9.0_cvmfs"
}
# Flip a coin to choose your version
my_version = choice(list(pipelines))
print("My version is:", my_version)

Show the inputs required by such application

In [ ]:
VipLauncher.show_pipeline(pipelines[my_version])

Build and print your inputs

In [ ]:
# Each subject contains 4 inputs files for the BraTS pipeline
input_settings = {
    "t1Image": [(input_dir / subject / "T1.nii.gz") for subject in subjects],
    "t1ceImage": [(input_dir / subject / "T1GD.nii.gz") for subject in subjects],
    "t2Image": [(input_dir / subject / "T2.nii.gz") for subject in subjects],
    "flImage": [(input_dir / subject / "T2-FLAIR.nii.gz") for subject in subjects],
    "appliOutputDir": subjects,
    "brainTumor": "1",
    "skullStrip": "1",
    "bundleAllResults": "0"
}

# Print your inputs
print("input_settings =",
    json.dumps(indent=2, 
               obj={ key: [str(v) for v in value] if isinstance(value, list) else str(value) for key, value in input_settings.items() } )
)

---
## Launch the Brain Tumor Segmentatiion

To control the size of your dataset, the pipeline will be launched in __teams__.
- *Please wait for the speakers to make the teams;*
- *Put your __team number__ instead of "X" in the following cell.*

In [ ]:
# PUT YOUR TEAM NUMBER
my_team = "miccai-team-X"

Launch the application

*Please execute the following cell __only once__ and wait until it the end.*

In [ ]:
# Stop if no team number is provided
if my_team == "miccai-team-X":
    raise ValueError("Please precise your team number in the cell above.")

# Launch the application
session = VipLauncher(
    output_dir = input_dir.parent / "outputs" / my_team / my_version,
    pipeline_id = pipelines[my_version],
    input_settings = input_settings
).run_session()

You should be invited to monitor your executions on the [VIP portal](https://vip.creatis.insa-lyon.fr).

When most executions are over within your team, you may proceed to **[Part II](2-analyze-outputs.ipynb)**.

---